In [ ]:

import seaborn as sns
import torch
import torch_geometric
import numpy as np
import pandas as pd
from utils import histogram_dicyclic_saver, plot_saved_data, make_path, save_tensors, prediction
import matplotlib.pyplot as plt
from dataset_gen import *
from train import training
import os
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# importing parameters from json config file
json_path = 'params_dicyclic.json'

with open(json_path) as f:
        params = json.load(f)
num_reps = params['num_reps']
n_max = params['n_max']
min_num_layers = params['min_num_layers']
max_num_layers = params['max_num_layers']
layers =[i for i in range(min_num_layers, max_num_layers+1)]
epochs = params['epochs']
hd = params['hd']
lr = params['lr']
k = params['k']
gap = params['gap']
experiment_type = params['exp_type']
early = params['early']
gnn_types = ['gconv_diff', 'gconv_glob']

In [ ]:
for num_l in layers:
    df_data = []
    print(f'Number of layers: {num_l}')
    for gnn_type in gnn_types:
        #selection of task: depends on gap and k
        if gap>0:
            path = make_path('cycles/'+experiment_type+'/extrap', gnn_type)
        else:
            path = make_path('cycles/'+experiment_type+'/extraction', gnn_type)
        config_json = pd.read_json(json_path, typ='series')
        config_json.to_csv(path+'config.txt')
        raw_data = []           
        for it in range(num_reps):    
            sep_vec = True   
            print(str(it+1)+'-th iteration')  

            #seed manual setting
            torch_geometric.seed.seed_everything(it*100+10)
            if gap>0:
                data_dict = get_complete_dataset_gap(n_max, gap)
            else:
                data_dict = extraction_task_dataset(n_max, k)
            
            model, raw_data = training(gnn_type, data_dict, 'dicyclic', hd, lr, num_l, 
                                       epochs, it, raw_data, device=device, path=path, early=early)
            
            data_loader = data_dict['test_loader_nobatch']
            iter_dl = iter(data_loader)

        df_data = prediction(model, hd, n_max, gap, gnn_type, iter_dl, sep_vec, df_data, device, path, num_l, it)
        df = pd.DataFrame.from_records(df_data)
        df.to_csv(path+'trial_nmax_'+str(n_max-2)+'_gap_'+str(gap+2)+'_'+str(num_l)+'_layers.csv')
        plot_saved_data(path = path, file_name = None, online=False, df=df, num_l = num_l)

    data = pd.DataFrame.from_records(raw_data)
    data.to_csv(path+'dicyclic_data')
    sns.lineplot(data=data, x='epoch', y='test_loss', hue='num_layers')
    
    plt.savefig(path+'test_loss/test_loss_dicyclic.png')
    plt.close()